In [1]:
import os
import shutil

# define the input folder path
input_folder = "E:\\Final-P\\Pneumonia-Detection-using-FastAI-main\\chest_xray\\train"

# define the output folder paths
non_virus_folder = "E:\\Final-P\\Pneumonia-Detection-using-FastAI-main\\chest_xray\\train\\non-viral"
virus_folder = "E:\\Final-P\\Pneumonia-Detection-using-FastAI-main\\chest_xray\\train\\viral"

# create the output folders if they don't exist
os.makedirs(non_virus_folder, exist_ok=True)
os.makedirs(virus_folder, exist_ok=True)

# loop through all the files in the input folder
for filename in os.listdir(input_folder):
    # check if the filename contains "bacteria" or "virus"
    if "bacteria" in filename.lower():
        # move the file to the bacteria folder
        shutil.move(os.path.join(input_folder, filename), non_virus_folder)
    elif "fungal" in filename.lower():
        # move the file to the virus folder
        shutil.move(os.path.join(input_folder, filename), non_virus_folder)
    elif "virus" in filename.lower():
        # move the file to the virus folder
        shutil.move(os.path.join(input_folder, filename), virus_folder)

In [2]:
from fastai.vision.all import *

lung_types = 'NORMAL','VIRUS','NONVIRAL'
path = Path('E:\\Final-P\\Pneumonia-Detection-using-FastAI-main\\chest_xray\\train')

# Define the data block
lungs = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128)
)

# Create dataloaders
dls = lungs.dataloaders(path)

# Create and fine-tune the VGG19 model
learn = cnn_learner(dls, vgg19_bn, metrics=accuracy)
learn.fine_tune(20)


c:\users\hp\appdata\local\programs\python\python39\lib\site-packages\fastai\vision\learner.py:301: UserWarning: `cnn_learner` has been renamed to `vision_learner` -- please update your code
  warn("`cnn_learner` has been renamed to `vision_learner` -- please update your code")


epoch,train_loss,valid_loss,accuracy,time
0,1.035490,0.673459,0.776606,36:05


epoch,train_loss,valid_loss,accuracy,time
0,0.646023,0.583496,0.777565,46:04
1,0.528572,0.500329,0.805369,38:49
2,0.451471,0.468191,0.819751,39:11
3,0.397291,0.521126,0.809204,52:30
4,0.351214,0.481324,0.815916,38:17
5,0.289303,0.558410,0.807287,39:10
6,0.238147,0.528837,0.813039,39:17
7,0.176856,0.624291,0.831256,40:07
8,0.155627,0.641734,0.820709,39:50
9,0.103683,0.607702,0.830297,1:46:07


In [18]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

ValueError: not enough values to unpack (expected 3, got 2)

In [12]:
interp = ClassificationInterpretation.from_learner(learn)
interp.print_classification_report()

ValueError: not enough values to unpack (expected 3, got 2)

In [16]:
_, pred_labels = learn.get_preds(dl=interp.dl, with_decoded=True)
true_labels = interp.dl.dataset.labels

accuracy = (pred_labels == true_labels).float().mean().item()

print("Accuracy:", accuracy)

AttributeError: labels

In [9]:
from fastai.vision.all import *

# Load the previously trained model
learn = load_learner("vggmodel.pkl")

# Create a ClassificationInterpretation object from the learner
interp = ClassificationInterpretation.from_learner(learn)

# Plot the confusion matrix
interp.plot_confusion_matrix()

# Get accuracy from the ClassificationInterpretation object
accuracy = interp.accuracy()

# Print accuracy
print("Accuracy:", accuracy)


ValueError: not enough values to unpack (expected 3, got 2)

In [19]:
# Export the trained model to a custom filename
learn.export('vggmodel.pkl')

path = Path()
path.ls(file_exts='.pkl')

# Load the exported learner from the custom filename
learn_inf = load_learner(path/'vggmodel.pkl')

# Test the model on individual images
learn_inf.predict('E:\\Final-P\\Pneumonia-Detection-using-FastAI-main\\chest_xray\\test\\NORMAL\\IM-0003-0001.jpeg')

('NORMAL', tensor(0), tensor([8.5146e-01, 1.9344e-06, 1.4853e-01]))

In [ ]:
learn_inf.predict('E:\\Final-P\\Pneumonia-Detection-using-FastAI-main\\chest_xray\\test\\PNEUMONIA\\person78_bacteria_381.jpeg')

In [ ]:
learn_inf.predict('E:\\Final-P\\Pneumonia-Detection-using-FastAI-main\\chest_xray\\test\\PNEUMONIA\\person44_virus_93.jpeg')

In [ ]:
import random
from PIL import Image

# Get the paths to 10 random test images
test_path = Path('E:\\Final-P\\Pneumonia-Detection-using-FastAI-main\\chest_xray\\test')
test_images = get_image_files(test_path)
random_images = random.sample(test_images, 10)

# Load the exported learner
learn_inf = load_learner(path/'export.pkl')

# Create a 2x5 table to display the images and their predictions
fig, ax = plt.subplots(2, 5, figsize=(12, 6))

# Loop through the 10 images and show the predicted labels
for i, img_path in enumerate(random_images):
    img = Image.open(img_path)
    img = img.resize((224, 224))
    predicted_label, _, _ = learn_inf.predict(img)
    
    # Display the image along with the predicted label
    row = i // 5
    col = i % 5
    ax[row, col].imshow(img)
    ax[row, col].axis('off')
    ax[row, col].set_title(f"{img_path.name}\n{predicted_label}", fontsize=8)

plt.tight_layout()
plt.show()

In [ ]:
import gradio as gr
from PIL import Image

def predict_image_vgg(img):
    # Convert numpy array to PIL Image
    img = Image.fromarray(img)

    # Resize the uploaded image
    img = img.resize((224, 224))

    # Make prediction using the VGG19 model
    pred, _, _ = learn_inf.predict(img)
    
    # Return the predicted label
    return pred

# Define the Gradio interface
iface_vgg = gr.Interface(8
    fn=predict_image_vgg,
    inputs=gr.components.Image(),
    outputs=gr.components.Textbox(label="Predicted Label"),
    title="Pneumonia Classification (VGG19)",
    description="Upload an image of a chest x-ray to classify it as normal, non-viral pneumonia, or viral pneumonia."
)

# Launch the interface
iface_vgg.launch(share=True)  # This will create a public link
